In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

from tqdm import tqdm

## 랜덤 시드 고정

In [2]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

## 전처리된 학습 데이터 불러오기

In [3]:
DATA_PATH = '/content/drive/MyDrive/KTO_Project/data/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

train_input = np.load(open(DATA_PATH + INPUT_TRAIN_DATA, 'rb'))
train_input = pad_sequences(train_input, maxlen=train_input.shape[1])
train_label = np.load(open(DATA_PATH + LABEL_TRAIN_DATA, 'rb'))
prepro_configs = json.load(open(DATA_PATH + DATA_CONFIGS, 'r'))

## 모델 하이파라미터 정의

In [4]:
model_name = 'rnn_classifier'
BATCH_SIZE = 128
NUM_EPOCHS = 5
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs = {'model_name': model_name,
         'vocab_size': prepro_configs['vocab_size'],
         'embedding_size': 100,
         'dropout_rate': 0.2,
         'lstm_dimension': 150,
         'dense_dimension': 150,
         'output_dimension':1}

## 모델 구현

In [5]:
class RNNClassifier(tf.keras.Model):

  def __init__(self, **kargs):
    super(RNNClassifier, self).__init__(name=kargs['model_name'])
    self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                      output_dim=kargs['embedding_size'])
    self.lstm_1_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'], return_sequences=True)
    self.lstm_2_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'])
    self.dropout = layers.Dropout(kargs['dropout_rate'])
    self.fc1 = layers.Dense(units=kargs['dense_dimension'], activation=tf.keras.activations.tanh)
    self.fc2 = layers.Dense(units=kargs['output_dimension'], activation=tf.keras.activations.sigmoid)
    
  def call(self, x):
    x = self.embedding(x)
    x = self.dropout(x)
    x = self.lstm_1_layer(x)
    x = self.lstm_2_layer(x)
    x = self.dropout(x)
    x = self.fc1(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

## 모델 생성

In [6]:
model = RNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.Precision(name='precision')\
                          ,tf.keras.metrics.Recall(name='recall')\
                          ,tf.keras.metrics.FalsePositives(name='false_positives')\
                          ,tf.keras.metrics.FalseNegatives(name='false_negatives')])

## 모델 학습

In [7]:
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

checkpoint_path = DATA_PATH + model_name + '\nweights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
  print("{} -- Foler already exists \n".format(checkpoint_dir))
else:
  os.makedirs(checkpoint_dir, exist_ok=True)
  print("{} -- Folder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, svae_weights_only=True)

history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_split=VALID_SPLIT)

SAVE_FILE_NM = 'weights.h5'
model.save_weights(os.path.join(DATA_PATH, SAVE_FILE_NM))

/content/drive/MyDrive/KTO_Project/data -- Foler already exists 

Epoch 1/5
448/448 [==============================] - 15s 12ms/step - loss: 0.1647 - precision: 0.9647 - recall: 0.9976 - false_positives: 2015.0000 - false_negatives: 131.0000 - val_loss: 0.0921 - val_precision: 0.9704 - val_recall: 0.9956 - val_false_positives: 186.0000 - val_false_negatives: 27.0000
Epoch 2/5
448/448 [==============================] - 4s 10ms/step - loss: 0.0797 - precision: 0.9769 - recall: 0.9954 - false_positives: 1296.0000 - false_negatives: 254.0000 - val_loss: 0.0844 - val_precision: 0.9766 - val_recall: 0.9935 - val_false_positives: 146.0000 - val_false_negatives: 40.0000
Epoch 3/5
448/448 [==============================] - 4s 10ms/step - loss: 0.0656 - precision: 0.9834 - recall: 0.9949 - false_positives: 929.0000 - false_negatives: 281.0000 - val_loss: 0.0843 - val_precision: 0.9769 - val_recall: 0.9935 - val_false_positives: 144.0000 - val_false_negatives: 40.0000
Epoch 4/5
448/448 [=========

In [8]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'
SAVE_FILE_NM = 'weights.h5'

test_input = np.load(open(DATA_PATH + INPUT_TEST_DATA, 'rb'))
test_input = pad_sequences(test_input, maxlen=test_input.shape[1])
test_label_data = np.load(open(DATA_PATH + LABEL_TEST_DATA, 'rb'))

In [9]:
model.load_weights(os.path.join(DATA_PATH, SAVE_FILE_NM))
model.evaluate(test_input, test_label_data)

663/663 [==============================] - 3s 5ms/step - loss: 0.0854 - precision: 0.9789 - recall: 0.9933 - false_positives: 437.0000 - false_negatives: 137.0000


[0.08541086316108704, 0.9789143800735474, 0.993292510509491, 437.0, 137.0]